# Helpers

In [ ]:
# Load "config.R" for utility functions. 
#Will also triggger loading of 
    
    # user_config.JSON (including key for project_config)
    # project_config.JSON
    # preprocessing_visualizations.R
    # preprocessing_functions.R

user <- "Jan" 
source("config.r")



#If certain packages not installed yet via requirements.txt, install them here via
# install.packages("package_name")

## Functions

In [ ]:
# All functions in preprocessing_functions.R are available now.

# Generate dataframes

### Prelim dfs for NA removal

In [ ]:
# Load dataframes preliminarily here to check which rows to keep/to remove. This will create the "df_nan" used later on.
# Define a priori the maximum number of NAs allowed per row
max_na_threshold <- 25

df_y <- data.table::fread("data/df_case_controls.txt", sep="\t")
df_covariates <- data.table::fread("data/df_covariates_1.7.txt", sep="\t")
df_diagnosis <- data.table::fread("data/df_diagnosis.txt", sep="\t")
df_blood <- data.table::fread("data/df_blood.txt", sep="\t")

#df_x_temp <- merge_dataframes(filter_par = FALSE)

dfs_to_merge <- list(df_covariates, df_diagnosis, df_blood, df_y)
df_x_temp <- Reduce(function(x, y) merge(x, y, by = "person_id", all = FALSE), dfs_to_merge)
all_na <- summarize_na(df_x_temp)
print(all_na)
write_xlsx(all_na, 'data/all_na.xlsx')


df_x_temp2 <- omit.NA(df_x_temp, max_na_threshold)
#str(df_all_cleaned)
df_nan <- df_x_temp2 %>% select("person_id") #df containing all rows with less than X NaN per row, used for subsetting later
write_xlsx(df_nan, "data/df_nan.xlsx")
summarize_na(df_x_temp2)

print(paste("Positive cases before removal of rows with >X NAs:", sum(df_x_temp$status)))
print(paste("Positive cases after removal of rows with >X NAs:", sum(df_x_temp2$status)))
                    
print(paste("Total rows before removal of rows with >X NAs:", nrow(df_x_temp)))
print(paste("Total rows after removal of rows with >X NAs:", nrow(df_x_temp2)))
                    
na_analysis <- analyze_na_thresholds(df_x_temp, max_threshold = 30, step = 5)
print(na_analysis)

# Optionally, you can write this to an Excel file

write_xlsx(na_analysis, "NA_threshold_analysis.xlsx")



In [ ]:
# Create an improved plot
ggplot(na_analysis, aes(x = NA_Threshold)) +
  geom_line(aes(y = Total_Rows, color = "Total Rows"), size = 1) +
  geom_point(aes(y = Total_Rows, color = "Total Rows"), size = 3) +
  geom_text(aes(y = Total_Rows, label = comma(Total_Rows)), 
            vjust = -0.5, size = 3, color = "black") +
  geom_line(aes(y = Positive_Cases * (max(Total_Rows) / max(Positive_Cases)), 
                color = "Positive Cases"), size = 1) +
  geom_point(aes(y = Positive_Cases * (max(Total_Rows) / max(Positive_Cases)), 
                 color = "Positive Cases"), size = 3) +
  geom_text(aes(y = Positive_Cases * (max(Total_Rows) / max(Positive_Cases)), 
                label = comma(Positive_Cases)),
            vjust = 1.5, size = 3, color = "black") +
  scale_y_continuous(name = "Total Rows",
                     sec.axis = sec_axis(~. * (max(na_analysis$Positive_Cases) / max(na_analysis$Total_Rows)), 
                                         name = "Positive Cases")) +
  scale_color_manual(values = c("Total Rows" = "blue", "Positive Cases" = "red")) +
  labs(title = "Impact of NA Threshold on Dataset Size",
       x = "NA Threshold",
       color = "Metric") +
  theme_minimal() +
  theme(legend.position = "bottom",
        axis.text.y.right = element_text(color = "red"),
        axis.title.y.right = element_text(color = "red"))

ggsave("visuals/NA_threshold_analysis_plot.png", width = 10, height = 6, dpi = 300)

## DOI

In [ ]:
df_nan <- read_xlsx("data/df_nan.xlsx") %>%  rename("eid" = "person_id")

dim(df_nan)
head(df_nan)

In [ ]:
df_y <- read_csv(file.path(data_path, "dataframes/df_y.csv")) %>%  rename("eid" = "person_id")
df_y

head(df_y)
dim(df_y)
sum(df_y$status)

## Covariates

### Import

In [ ]:
df_covariates <- fread(file.path(data_path, "dataframes/df_covariates_1.7.txt"), sep="\t") %>%  rename("eid" = "person_id")
df_covariates

df_covariates$SEX <- factor(df_covariates$SEX, levels = c("Female", "Male"), labels = c("0", "1"))
df_covariates$'Ever smoked' <- factor(df_covariates$'Ever smoked', levels = c("No", "Yes"), labels = c("0", "1"))

dim(df_covariates)
head(df_covariates)

covariates_list <- colnames(df_covariates)[-1]

#check_missing_positives(df_covariates)
str(df_covariates)
summary(df_covariates)

### Cut to physiol. limits

In [ ]:
mapper <- read_excel(file.path(data_path, "unit_mapper_ukb_aou.xlsx"), sheet = "Mapper")
result <- limit_df(df_covariates, mapper)
df_covariates <- result$df
summary(df_covariates)

## Diagnosis

In [ ]:
df_diagnosis <- read_csv(file.path(data_path, "dataframes/df_diagnosis_filtered.csv")) %>% rename("eid" = "person_id") 

dim(df_diagnosis)
head(df_diagnosis)

diagnosis_list <- colnames(df_diagnosis)[-1]


diag_na <- summarize_na(df_diagnosis, rule=TRUE)
print(diag_na)
check_missing_positives(df_diagnosis)


## Blood

### Load and impute

In [ ]:
# Create a single df blood from the separate timeframes

df_blood_pre2010  <- read_csv(file.path(data_path, "dataframes/df_blood_pre2010_i.csv"))
df_blood2010_2015 <- read_csv(file.path(data_path, "dataframes/df_blood2010_2015_i.csv"))
df_blood2015_2020 <- read_csv(file.path(data_path, "dataframes/df_blood2015_2020_i.csv"))
df_blood2020_2025 <- read_csv(file.path(data_path, "dataframes/df_blood2015_2020_i.csv"))

df_blood <- rbindlist(list(
  as.data.table(df_blood_pre2010),
  as.data.table(df_blood2010_2015),
  as.data.table(df_blood2015_2020)
), use.names = TRUE, fill = TRUE)

df_blood <- df_blood %>% select(-(c("SEX", "AGE", "BMI"))) %>%  rename("eid" = "person_id")# %>%
 # semi_join(df_nan, by = "eid")


head(df_blood)
dim(df_blood)


In [ ]:
map_and_align <- function(df, mapper_df) {
  
  # Create a named vector for renaming, excluding NA mappings
  rename_vector <- setNames(mapper_df$column_ukb, mapper_df$column_aou)
  rename_vector <- rename_vector[!is.na(rename_vector) & !is.na(names(rename_vector))]
    
    # Keep only mappings for columns actually in df
  rename_vector <- rename_vector[names(rename_vector) %in% names(df)]
  
  # Identify columns to remove (those with NA in column_ukb or column_aou)
  columns_to_remove <- unique(c(
    mapper_df$column_aou[is.na(mapper_df$column_ukb)],
    mapper_df$column_aou[is.na(mapper_df$column_aou)]
  ))
  columns_to_remove <- columns_to_remove[!is.na(columns_to_remove)]  # Remove NA values
  
  # Rename columns
  df_renamed <- df %>%
    rename_with(~ ifelse(.x %in% names(rename_vector), rename_vector[.x], .x), everything())
  
  # Remove columns with NA mappings
  df_final <- df_renamed %>%
    select(-any_of(intersect(columns_to_remove, names(df_renamed))))
  
  # Identify columns that weren't renamed (no mapping found)
  unmapped_cols <- setdiff(names(df_final), mapper_df$column_ukb[!is.na(mapper_df$column_ukb)])
  
  # Print summary
  cat("Summary of map_and_align:\n")
  cat(sprintf("- %d columns renamed\n", sum(names(df) %in% names(rename_vector))))
  cat(sprintf("- %d columns identified for removal due to NA mapping\n", length(columns_to_remove)))
  cat(sprintf("- %d columns actually removed\n", ncol(df_renamed) - ncol(df_final)))
  cat(sprintf("- %d columns left unmapped\n", length(unmapped_cols)))
  
  # Print removed columns
  if (length(columns_to_remove) > 0) {
    cat("\nColumns identified for removal due to NA mapping:\n")
    print(columns_to_remove)
  }
  
  # Print unmapped columns
  if (length(unmapped_cols) > 0) {
    cat("\nColumns not renamed (no mapping found):\n")
    print(unmapped_cols)
  }
  
  return(df_final)
}

In [ ]:
mapper <- read_excel(file.path(data_path, "unit_mapper_ukb_aou.xlsx"), sheet = "Mapper")

df_blood <- map_and_align(df_blood, mapper_df=mapper)

dim(df_blood)
head(df_blood)

blood_list <- colnames(df_blood)[-1]
#df_blood <- impute_continuous(df_blood)
summarize_na(df_blood)
vec_blood <- setdiff(colnames(df_blood), "eid")
vec_blood


### Blood unit conversion and outlier adjustment

In [ ]:
mapper <- read_excel(file.path(data_path, "unit_mapper_ukb_aou.xlsx"), sheet = "Mapper")
conversion_table <- read_xlsx("data/Master_Table_JC.xlsx", sheet="Mapper")
df_blood <- convert_units(df_blood, mapper)

In [ ]:
df_blood_adjusted <- adjust_outlier_to_ukb(df_blood, conversion_table)

In [ ]:
df_summary_ukb <- read_xlsx("data/UKB_MinMax.xlsx")
#df_summary_ukb
df_summary_aou_blood <- summarize_continuous_columns(df_blood_adjusted)
#df_summary_aou_blood

df_summary_blood <- inner_join(df_summary_ukb, df_summary_aou_blood, by= "column", suffix = c("_ukb", "_aou")) %>%
  select(column, sort(setdiff(names(.), "column")))
df_summary_blood
write_xlsx(df_summary_blood, "tables/df_summary_blood_ukb_aou.xlsx")

### Check blood_nas individually

In [ ]:
df_blood_y <- inner_join(df_blood, df_y, by = "eid") %>% filter(status == 1)

dim(df_blood_y)
head(df_blood_y)

blood_na <- summarize_na(df_blood_y)
print(blood_na)


## Genetics

In [ ]:
df_snp <- data.table::fread("data/SNPs.raw", sep="\t")

df_snp <- df_snp %>%
    select(-FID, -PAT, -MAT, -SEX, -PHENOTYPE) %>%
    rename(person_id = IID)

dim(df_snp)
head(df_snp)

snp_list <- colnames(df_snp)[-1]


In [ ]:
colnames(df_all)

## Merge, normalize and save

### Merge X

In [ ]:
df_x_all_raw <- merge_dataframes(filter_par = FALSE)

dim(df_x_all_raw)
head(df_x_all_raw)
colnames(df_x_all_raw)

In [ ]:
df_y

In [ ]:
str(df_y$year_of_diag)
str(df_x_all_raw$year)

In [ ]:
df_x_all_raw$year

### Merge with y

In [ ]:
df_x_all_raw
colnames(df_x_all_raw)

df_y
colnames(df_y)

In [ ]:
#df_all <- df_x_all_raw %>% inner_join(df_y, by = "eid")  #Merge with DOI cases

df_all <- assign_labels_with_horizon(df_x_all_raw, df_y, horizon = 5)


dim(df_all)
head(df_all)
colnames(df_all)


In [ ]:
df_all_overview <- df_all %>%
  group_by(status) %>%
  slice_sample(n = 5) %>%
  ungroup() %>%
  select(eid, year, year_of_diag, year_diff, status)

# Print the preview
print(df_all_overview)

In [ ]:
DOI

In [ ]:
# Plot bar chart of included and discarded cases
plot_included_discarded_cases(
  df = df_all,
  year_col = "year_of_diag",
  target_col = "status",
  include_value = 1,
  discard_value = 2,
  base_size = 30
)


# Remove cases with status 2 (discarded because event (=DOI) before assessment visit)
df_all <- subset(df_all[df_all$status!=2])

In [ ]:
# Remove rows with status == 2 for further analysis
df_all <- df_all[df_all$status != 2, ]

# Check summary of status column
sum(df_all$status)

dim(df_all)

In [ ]:
df_par <- df_all

## Prepare PAR Dataframe

In [ ]:
# Read PAR requirements
par_index <- read_excel("data/Master_Table_JC.xlsx", sheet= "Patients at risk")

par_index

In [ ]:
df_par <- df_par %>%
  mutate(
    Elevated_AST = if_else(SEX == "0" & `Aspartate aminotransferase` > 35 | SEX == "1" & `Aspartate aminotransferase` > 50, 1, 0),
    Elevated_ALT = if_else(SEX == "0" & `Alanine aminotransferase` > 35 | SEX == "1" & `Alanine aminotransferase` > 50, 1, 0),
  ) %>%
  mutate(Elevated_Liver_Enzymes = if_else(Elevated_AST == 1 | Elevated_ALT == 1, 1, 0))
  
df_par

In [ ]:
#Adjust your par subset to the preferred groups you would like to set as inclusion criteria
par_subset <- c("CLD", "Cirrhosis", "Viral Hepatitis", "Blood_Parameters")

df_par <- filter_rows_with_pos_entries(df_par)

### Normalization execution df_all

In [ ]:
vec_covariates <- c("MultipleDeprivationIndex", "Pack years", "Waist circumference", # Columns for "simple" Minmax in covariates
                  "Weight", "Standing height", "Alk_g_d", "Bloodpressure_sys", "Bloodpressure_dia", "BMI", "AGE")

vec_blood <- setdiff(colnames(df_blood), "eid")

vec_all <- unique(c(vec_covariates, vec_blood)) 
vec_all




df_x_all_normalized <- normalize_data_ukb(df_all, conversion_table, vec_blood) %>%
    normalize_data_aou(vec_covariates) %>%
    select(-(c("status", "year_of_diag", "year_diff")))
    

head(df_x_all_normalized)
str(df_x_all_normalized)
summary(df_x_all_normalized)

### Normalization execution df_all

In [ ]:
vec_covariates <- c("MultipleDeprivationIndex", "Pack years", "Waist circumference", # Columns for "simple" Minmax in covariates
                  "Weight", "Standing height", "Alk_g_d", "Bloodpressure_sys", "Bloodpressure_dia", "BMI", "AGE")

vec_blood <- setdiff(colnames(df_blood), "eid")

vec_all <- unique(c(vec_covariates, vec_blood)) 
vec_all




df_x_par_normalized <- normalize_data_ukb(df_par, conversion_table, vec_blood) %>%
    normalize_data_aou(vec_covariates) %>%
    select(-(c("status", "year_of_diag", "year_diff")))
    

head(df_x_par_normalized)
str(df_x_par_normalized)
summary(df_x_par_normalized)

### Sort out dataframes and remove columns


### Check y

In [ ]:
#All
df_y_all <- df_all %>% select("eid", "status", "year_diff", "year_of_diag", "year", "person_id_year")


head(df_y_all)
str(df_y_all)
sum(df_y_all$status)
nrow(df_y_all) == nrow(df_x_all_normalized)

In [ ]:
#PAR
df_y_par <- df_par %>% select("eid", "status", "year_diff", "year_of_diag", "year", "person_id_year")


head(df_y_par)
str(df_y_par)
sum(df_y_par$status)
nrow(df_y_par) == nrow(df_x_par_normalized)

### Export preparation

In [ ]:
# ALL
col_subset <- "basic"
layer <- "outer"

# Create a dated folder name (e.g., "15_04_2025")
today <- format(Sys.Date(), "%d_%m_%Y")
output_dir <- paste0("data/", today)

# Create the directory if it doesn't exist
if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE)
}


### Export df_all

In [ ]:
# All

subset_name <- "all"
write_csv(df_x_all_normalized, file=paste0(output_dir, "/X_", layer, "_", col_subset, "_", subset_name, ".csv"))
write_csv(df_y_all, file=paste0(output_dir, "/y_", layer, "_", col_subset, "_", subset_name, ".csv"))
write_csv(df_all, file=paste0(output_dir, "/df_all_", layer, "_", col_subset, "_", subset_name, ".csv"))

### Export df_par

In [ ]:
# PAR

subset_name <- "par"
write_csv(df_x_par_normalized, file=paste0(output_dir, "/X_", layer, "_", col_subset, "_", subset_name, ".csv"))
write_csv(df_y_par, file=paste0(output_dir, "/y_", layer, "_", col_subset, "_", subset_name, ".csv"))
write_csv(df_par, file=paste0(output_dir, "/df_par_", layer, "_", col_subset, "_", subset_name, ".csv"))

In [ ]:
# In case you start here, load df_all again
output_dir <- "data/24_04_2025"
col_subset <- "basic"
subset_name_input <- "all"
layer <- "outer"

#df_all <- read.csv(file=paste0(output_dir, "/df_all_", layer, "_", col_subset, "_", subset_name_input, ".csv")) %>% select(-starts_with("..."))

In [ ]:
head(df_all)
dim(df_all)

In [ ]:
df_par <- filter_rows_with_pos_entries(df_all) # Filter the "population at risk (par) by a prespecified if required

head(df_par)
dim(df_par)
summary(df_par$status)

# Summary tables

## Installing dependencies

In [ ]:
#install.packages("gtsummary")
#install.packages("flextable")
#install.packages("officer")
#remotes::install_version("glue", version = "1.8.0")
install.packages("cardx")

## Function load

In [ ]:
library(gtsummary)
library(flextable)
library(officer)
library(cardx)
library(gt)

create_binary_list <- function(df) {
    binary_cols <- sapply(df, function(x) all(x %in% c(0, 1)))
    names <- names(df)[binary_cols]

    return(names)
}

#Helper function to mask items with absolute < 20
categorical_stat <- function(x) {
    n <- sum(x == "1" | x == 1, na.rm = TRUE)
    N <- sum(!is.na(x))
    count_display <- if(n < 20) "<20" else as.character(n)
    sprintf("%s (%.1f%%)", count_display, n/N * 100)
}                          
                          
                          
# https://www.danieldsjoberg.com/gtsummary/articles/tbl_summary.html

create_table <- function(df_tbl, table_name, table_name_prefix="All", export_RDS=FALSE, head_only=FALSE, remove_SEX=TRUE, enforced_order=FALSE, biobank_key=biobank_key, create_binary_table=FALSE, adjust_p_values=TRUE, column_order = FALSE) {
    
    categorical_stat_fmt <- "{n} ({p}%)"
  
    if (remove_SEX) {
        df_tbl <- df_tbl %>% select(-SEX)
    }
  
    if (!isFALSE(enforced_order)) {
        df_tbl <- df_tbl %>% select(status, all_of(enforced_order))
    }
  
    if (isFALSE(enforced_order)) {
        names <- sort(colnames(df_tbl))
        df_tbl <- df_tbl %>% select(all_of(names)) 
    }
  
    if (head_only) {
        df_tbl <- df_tbl %>% select(1:5, status)
    }
  
    if (create_binary_table) {
        names <- create_binary_list(df_tbl)
        value_list <- lapply(names, function(name) {
            formula <- as.formula(paste0("`", name, "` ~ '1'"))
            environment(formula) <- emptyenv()
            return(formula)
        })

        label_list <- lapply(names, function(name) {
            formula <- as.formula(paste0("`", name, "` ~ '", name, "'"))
            environment(formula) <- emptyenv()
            return(formula)
        })
    } else {
        value_list <- NULL
    }
  
    if (!isFALSE(column_order)) {
        df_tbl$status <- factor(df_tbl$status, levels = column_order)
    }
    
    distinct_status_values <- df_tbl %>% distinct(status) %>% pull(status) %>% sort()
  
    continuous_columns <- c("Pack years", "Basophill (%)", "Eosinophill (%)", "Monocyte percentage", "Neutrophill count", "Total protein")
    continuous_columns_present <- continuous_columns[continuous_columns %in% colnames(df_tbl)]
    type_list <- if (length(continuous_columns_present) > 0) {
        lapply(continuous_columns_present, function(col) {
            as.formula(paste0("`", col, "` ~ 'continuous'"))
        })
    } else {
        NULL
    }
  
    
    
Table_gtsummary <- df_tbl %>%
    tbl_summary(
        by = status,
        type = type_list,
        value = value_list,
        label = label_list,
        statistic = list(
            all_continuous() ~ "{mean} (±{sd})",
            all_categorical() ~ categorical_stat_fmt
        ),
        digits = all_continuous() ~ 1,
    ) %>%
    add_overall() %>%
    modify_table_body(
            ~ .x %>%
                mutate(
                    across(starts_with("stat_"), ~ case_when(
                        # Mask counts <20 in categorical columns and remove percentage
                        str_detect(., "^\\d+,?\\d* \\(") & 
                        as.numeric(gsub(",", "", str_extract(., "^\\d+,?\\d*"))) < 20 ~ 
                            str_replace(., "^\\d+,?\\d* \\(.*?\\)", "<20"),  # Removes count and percentage
                        TRUE ~ .
                    ))
                )
        ) %>%
    add_p(
        test = list(
            all_continuous() ~ "t.test",
            all_categorical() ~ "chisq.test"
        )
    ) %>%
        
        bold_labels() %>%
        italicize_levels() %>%
        bold_p() %>%
        modify_header(update = list(
            stat_0 ~ "**Overall** \n \n N = {N}",
            !!sym(paste0("stat_", 1)) ~ paste0("**", distinct_status_values[1], "** \n\n n = {n}"),
            !!sym(paste0("stat_", 2)) ~ paste0("**", distinct_status_values[2], "** \n\n n = {n}")
        ), text_interpret = "md")
  
    if (adjust_p_values) {
        Table_gtsummary <- Table_gtsummary %>%
            add_q(method = "bonferroni")
    }
  
    Table_gt <- as_gt(Table_gtsummary) %>%
      fmt_number(
          columns = c(-contains("p.value"), -contains("q.value")),  # Exclude p-value and q-value columns
          decimals = 2,
          locale = "en"  # Locale enforces '.' for decimals and ',' for thousands
    )
    print(Table_gt)
  
    if (export_RDS) {
        saveRDS(Table_gtsummary, file = paste0("tables/", table_name, ".RDS"))
    }
  
    gt::gtsave(Table_gt, filename = paste0("tables/", table_name, ".html"))
  
    #Table_gtsummary %>%
     #   as_flex_table() %>%
      #  save_as_docx(path = paste0("tables/", table_name, "_", biobank_key, ".docx"))
}
     
                          
                          
                          
                          
split_create_merge_tables <- function(df, feature, table_name, enforced_order=FALSE, head_only=FALSE, remove_SEX=TRUE, export_RDS=FALSE, create_binary_table = FALSE, adjust_p_values=TRUE) {
    
  
    # Split the dataframe by the specified feature
    split_dfs <- split(df, df[[feature]])
  
    if (!isFALSE(enforced_order)) {
        enforced_order <- table1_order[table1_order != feature] #Remove feature to stratify from the order list
    }
  
    # Iterate over each split dataframe
    table_files <- c() # To store filenames of the saved tables for potential merging
  
    for (split_name in names(split_dfs)) {
        # Define table name based on prefix and split name
        merged_name <- paste(table_name, feature, split_name, sep="_")
        #table_files <- c(table_files, paste0(table_name, "_", Sys.Date(), ".RDS"))

        # Use create_table to generate and save each table
    
        create_table(df_tbl = split_dfs[[split_name]], 
                     table_name = merged_name, 
                     export_RDS = export_RDS, 
                     head_only = head_only, 
                     remove_SEX = remove_SEX, 
                     enforced_order = enforced_order,
                     create_binary_table = create_binary_table,
                     adjust_p_values = adjust_p_values)
    }
  
    # Optionally merge the tables if more than one split
    #if(export_RDS) {
    #  tab_spanner <- names(split_dfs)
    #  merge_saved_tables(table_files, project_path, tab_spanner)
    #}

    #gt::gtsave(Table_1_stratified, filename = paste0(project_path, "/tables/Table_stratified", table_name, "_", biobank_key, "_", Sys.Date(), ".html"))
}   
                          
                          
                          
import_merge_tables <- function(table_name, feature, levels,  tab_spanner) {
    
  
    # Initialize an empty list to store the imported tables
    imported_tables <- list()
  
    # Loop through each level and import the corresponding RDS file
    for (level in levels) {
        file_path <- paste0("tables/", table_name, "_", feature, "_", level, ".RDS")
        print(file_path)

        imported_table <- readRDS(file_path)
        #imported_table <- imported_table %>% filter(Variable != feature)
        imported_tables[[level]] <- imported_table
    }

    # Merge the imported tables
    merged_table <- tbl_merge(
        tbls = imported_tables,
        tab_spanner = levels
    )
    
    print(merged_table)
    merged_table <- as_gt(merged_table)
    gt::gtsave(merged_table, filename = paste0("tables/", table_name, "_", feature, ".html"))
    # Return the merged table
    return(merged_table)
}


## Load data

In [ ]:
df_all <- read_csv("data/24_04_2025/df_all_outer_basic_all.csv") %>% select(-starts_with("..."))
head(df_all)
colnames(df_all)

columnmapper <- read_excel("data/columnmapper_ukb_aou.xlsx")
blood_list <- columnmapper$column_ukb[columnmapper$source_df=="df_blood"]
diagnosis_list <-columnmapper$column_ukb[columnmapper$source_df=="df_diagnosis"]

## Define labels, orders, load additional data

In [ ]:
label_list <- list(
  AGE = "Age [years]",
  location_name = "Location",
  location_country = "Country",
  Family_diabetes = "Family Diabetes",
  DM = "Diabetes mellitus",
  Alk_g_d = "Alcohol [g/d]",
  High_Alk = "High Alcohol Consumption",
  Path_Alk = "Pathological Alcohol Consumption",
  BMI_cat = "BMI Categories",
  Bloodpressure_sys = "Bloodpressure sys. [mmHg]",
  Weight = "Weight [kg]",
  'Standing height' = "Standing height [cm]",
  'Waist circumference' = "Waist circumference [cm]",
  race_ethnicity = "Self-reported ethnicity"
)

table1_order <- c(
  "AGE", "SEX", "BMI", "Waist circumference", 
  "Weight", "Standing height", "race_ethnicity", 
  "MultipleDeprivationIndex", "Bloodpressure_sys", "Medication", "DM",
  "Family_diabetes", "Pack years", "Alk_g_d" # Continue as necessary
)

#df_all <- df_x_all_raw %>% inner_join(df_y, by = "eid")
df_ethnicity <- read_csv("data/df_ethnicity.csv")

df_table <- df_all %>% merge(df_ethnicity %>% select(eid, race_ethnicity), by = "eid")

print(length(df_table[df_table$status==2]))

df_table <- df_table  %>%
  filter(status != 2) %>%
  arrange(eid, desc(status)) %>%  # 1 comes before 0 because descending
  group_by(eid) %>%
  slice_head(n = 1) %>%  # Take the first row (after ordering by status)
  ungroup()

df_table <- df_table %>%
  mutate(
    SEX = case_when(
      SEX == "0" ~ "Female",
      SEX == "1" ~ "Male",
      TRUE ~ as.character(SEX)
    ),
    status = case_when(
      status == 0 ~ paste0("No ", DOI),    # Removed quotes since status is numeric
      status == 1 ~ DOI,       # Removed quotes since status is numeric
      TRUE ~ as.character(status)  # Convert to character for consistency
    )
  )

nrow(df_table)
dim(df_table)
head(df_table)

In [ ]:
table(df_table$SEX)

## Create Tables

### Non-stratified Tables

In [ ]:
df_tbl_1 <- df_table %>%
  select(!any_of(c(blood_list, setdiff(diagnosis_list, "DM"), "eid", "Date of assessment")))
  #elect(!any_of(c(blood_list, diagnosis_list, "eid", "Date of assessment", snp_list)))
create_table(df_tbl_1, "Table 1", export_RDS=FALSE, head_only=FALSE, remove_SEX=FALSE,  enforced_order=table1_order)

In [ ]:
df_tbl_blood <- df_table %>%
  select(any_of(c("status", "SEX", blood_list)))

create_table(df_tbl_blood, "Table Blood", export_RDS=FALSE, head_only=FALSE)


In [ ]:
df_tbl_icd <- df_table %>%
  select(any_of(c("status", "SEX", diagnosis_list)))

# Remove codes with 0 cases
#dim(df_tbl_icd)
#missing_codes <- names(df_tbl_icd[,3:ncol(df_tbl_icd)])[colSums(df_tbl_icd[,3:ncol(df_tbl_icd)]) == 0]
#print(missing_codes) #Dengue fever, oesophageal varices, yellow fever
#df_tbl_icd <- df_tbl_icd %>% select(-any_of(missing_codes))
#dim(df_tbl_icd)

Table_ICD <- create_table(df_tbl_icd, "Table ICD", export_RDS=TRUE, head_only=FALSE, create_binary_table = FALSE)


In [ ]:
df_tbl_snp <- df_table %>%
  select(any_of(c("status", "SEX", snp_list)))

create_table(df_tbl_snp, "Table SNP", export_RDS=FALSE, head_only=FALSE)


### Stratified tables (after sex)

In [ ]:
# Table 1 Sex-Stratified

df_tbl_1 <- df_table %>%
  select(!any_of(c(blood_list, setdiff(diagnosis_list, "DM"), "eid", "Date of assessment")))
split_create_merge_tables(df_tbl_1, table_name="Table1", feature="SEX", enforced_order=table1_order, remove_SEX=TRUE, export_RDS=TRUE)
Table_1_stratified <- import_merge_tables(table_name= "Table1", feature="SEX", levels = c("Female", "Male"))

In [ ]:
# Table ICD Sex-Stratified

df_tbl_icd <- df_table %>%
  select(any_of(c("status", "SEX", diagnosis_list)))


split_create_merge_tables(df_tbl_icd, table_name="Table_ICD", feature="SEX", enforced_order=FALSE, remove_SEX=TRUE, head_only=FALSE, export_RDS=TRUE)
Table_ICD_stratified <- import_merge_tables(table_name= "Table_ICD", feature="SEX", levels = c("Female", "Male"))



In [ ]:
# Table Blood Sex-Stratified

df_tbl_blood <- df_table %>%
  select(any_of(c("status", "SEX", blood_list)))


split_create_merge_tables(df_tbl_blood, table_name="Table_Blood", feature="SEX", enforced_order=FALSE, remove_SEX=TRUE, head_only=FALSE, export_RDS=TRUE)
Table_Blood_stratified <- import_merge_tables(table_name= "Table_Blood", feature="SEX", levels = c("Female", "Male"))



## Compare AOU with UKB Predictions (only run this after modelling)

In [ ]:
pred_val

In [ ]:
df_tbl_TP <- df_table %>%
  select(!any_of(c(blood_list, setdiff(diagnosis_list, c("DM","Liver cirrhosis")), "Date of assessment", "status", "race_ethnicity")))

df_tbl_TP <- df_tbl_TP %>%
    left_join(df_ethnicity %>% select(eid, ethnicity_ukb_aligned), by = "eid")


col_subset <- "Model_TOP15"
row_subset <- "all"

pred_val <- read_excel("combined_output/val/Prediction_values_combined.xlsx", sheet= paste0(row_subset, "_", col_subset))  #Change sheet as desired by changing col/row subset variables


pred_val <- inner_join(pred_val, df_tbl_TP, by="eid")

pred_val <- pred_val %>% select(-("eid"))

 

#Define thresholds and create classes
low_threshold <- 0.55

pred_val <- pred_val %>%
  mutate(
    TP = if_else(status == 1 & y_pred > low_threshold, 1, 0),    #Add columns with TP TN etc each as boolean
    TN = if_else(status == 0 & y_pred < low_threshold, 1, 0),
    FP = if_else(status == 0 & y_pred > low_threshold, 1, 0),
    FN = if_else(status == 1 & y_pred < low_threshold, 1, 0)
  )

pred_val <- pred_val %>%
  mutate(status = case_when(
      TP == 1 ~ "TP",
      TN == 1 ~ "TN",
      FP == 1 ~ "FP",
      FN == 1 ~ "FN",
      TRUE ~ NA_character_
    )
  )

pred_val$Medication <- factor(pred_val$Medication, 
                               levels = sort(unique(pred_val$Medication)))

pred_val <- pred_val %>%
    mutate(ethnicity_ukb_aligned = factor(ethnicity_ukb_aligned,
                                          levels = c("Asian", "Black", "Caucasian", "Latinx", "Other/Unknown"))) %>%
    rename(Ethnicity = ethnicity_ukb_aligned)


In [ ]:
pred_val

## Variable Preparation

In [ ]:

table_tp_order <- c(
  "y_pred", "AGE", "SEX", "Ethnicity",  "BMI", "Waist circumference", 
  "Weight", "Standing height",
  "MultipleDeprivationIndex", "Bloodpressure_sys", "Medication", "DM",
  "Family_diabetes", "Pack years", "Alk_g_d", "Liver cirrhosis" # Continue as necessary
)


label_list <- list(
  AGE = "Age [years]",
  location_name = "Location",
  location_country = "Country",
  Family_diabetes = "Family Diabetes",
  DM = "Diabetes mellitus",
  Alk_g_d = "Alcohol [g/d]",
  High_Alk = "High Alcohol Consumption",
  Path_Alk = "Pathological Alcohol Consumption",
  BMI_cat = "BMI Categories",
  Bloodpressure_sys = "Bloodpressure sys. [mmHg]",
  Weight = "Weight [kg]",
  'Standing height' = "Standing height [cm]",
  'Waist circumference' = "Waist circumference [cm]",
  y_pred = "Prediction Score",
  Alk_g_d = "Alcohol [g/d]",
  Ethnicity = "Ethnicity"
)

head_only = FALSE

table_name = paste0("All TP TN Options_threshold_", low_threshold, "_AOU")

In [ ]:
#Create the TP FN Table for AOU and save as RDS
create_table(pred_val, table_name, export_RDS=TRUE, head_only=FALSE, remove_SEX=FALSE,  enforced_order=table_tp_order, column_order = c("FN", "TP", "FP", "TN"))




In [ ]:
# Function to merge the two RDS files
## Make sure that not only the visible names align, but also the actual names of the underlying columns of the dataframe, otherwise this does not work (and the rows will be attached below)
fuse_biobank_tables <- function(table_name, biobanks = c("UKB", "AOU")) {
  low_threshold <- get("low_threshold", envir = .GlobalEnv)
  
  # Initialize an empty list to store the imported tables
  imported_tables <- list()
  
  # Loop through each level and import the corresponding RDS file
  for (biobank in biobanks) {
    file_path <- paste0("tables/", table_name, "_", low_threshold, "_", biobank, ".RDS")
    print(file_path)
    
    imported_table <- readRDS(file_path)
    # Set the locale to ensure consistent number formatting
    Sys.setlocale("LC_NUMERIC", "en_US.UTF-8")
    imported_tables[[biobank]] <- imported_table
  }
  
  # Merge the tables with modified options
  merged_table <- tbl_merge(
    tbls = imported_tables,
    tab_spanner = biobanks
  )
  
  # First print the gtsummary object
  print(merged_table)
  
  # Save as HTML using as_flex_table
  html_path <- paste0("tables/", table_name, "_merged_", paste(biobanks, collapse="_"), "_threshold_", low_threshold, ".html")
  
  # Try to save using flextable
  merged_table_gt <- as_gt(merged_table)
  gt::gtsave(merged_table_gt, filename = html_path)
  
  return(merged_table)
}

# Test the function
Table_TPFN_stratified <- fuse_biobank_tables(
  table_name = "All TP TN Options_threshold", 
  biobanks = c("UKB", "AOU")
)

New heading

# Create literature benchmark scores

In [ ]:
df_all <- read_csv("data/24_04_2025/df_all_outer_basic_all.csv") %>% select(-starts_with("..."))
head(df_all)

In [ ]:
df_early_cirrhosis <- read_csv("data/df_early_cirrhosis.csv")

df_early_cirrhosis <- df_early_cirrhosis %>%
    rename(eid = person_id) %>%
    merge(df_all %>% select(eid), by = c("eid"), all=TRUE) %>%
    mutate(cirrhosis = coalesce(cirrhosis, 0))

## AMAP Score

In [ ]:
df_amap <- df_all %>%
    select(c("eid", "person_id_year", "SEX", "AGE", "Platelet count", "Albumin", "Total bilirubin", "status")) %>%
    rename(
    bilirubin = `Total bilirubin`,
    platelet_count = `Platelet count`,
    albumin = `Albumin`
  )

df_amap$SEX <- as.character(df_amap$SEX)
df_amap$SEX <- as.numeric(df_amap$SEX)

summary(df_amap)
df_amap$aMAP <- ((df_amap$AGE * 0.06 + df_amap$SEX * 0.89 + 0.48 * ((log10(df_amap$bilirubin) * 0.66) + (df_amap$albumin * -0.085)) - 0.01 * df_amap$platelet_count) + 7.4) / 14.77 * 100
head(df_amap)
df_amap <- df_amap %>% 
  select(c("eid", "person_id_year", "aMAP", "SEX", "status")) %>%
  mutate(
    aMAP = as.numeric(aMAP) / 100,  # Ensure numeric before division
    aMAP = ifelse(is.infinite(aMAP), NA, aMAP)  # Replace Inf with NA
  )
summary(df_amap)

df_amap <- df_amap %>%
  mutate(
  aMAP = as.numeric(round(aMAP, 3))
  )  # Round to 2 decimal places)

head(df_amap, 10)
str(df_amap$'Total')
summary(df_amap)
write.csv(df_amap, file=paste("data/df_amap.csv", sep=''))

## Fib-4

In [ ]:
df_fib4 <- df_all %>% select("eid", "person_id_year", "AGE", AST = `Aspartate aminotransferase`,
                     ALT = `Alanine aminotransferase`,
                     PLT = `Platelet count`)

df_fib4 <- df_fib4 %>%
  mutate(
    FIB4_orig = (AGE * AST) / (PLT * (ALT * 0.5))
  )

df_fib4 <- adjust_outliers(df_fib4, "FIB4_orig")


# Calculate normalized FIB-4 score (0 to 1)
df_fib4 <- df_fib4 %>%
  mutate(
    FIB4 = (FIB4_orig - min(FIB4_orig, na.rm = TRUE)) / (max(FIB4_orig, na.rm = TRUE) - min(FIB4_orig, na.rm = TRUE))
  )
head(df_fib4)
sum(is.na(df_fib4$FIB4))
summary(df_fib4)
hist(df_fib4$FIB4_orig)

In [ ]:
df_apri <- df_all %>% select("eid", "person_id_year", "SEX", AST = `Aspartate aminotransferase`,
                     PLT = `Platelet count`)

summary(df_apri)

df_apri <- df_apri %>%
  mutate(
    ULN_AST = case_when(
      SEX == "0" ~ 35,
      SEX == "1" ~ 50,
      TRUE ~ NA_real_
    ),
    APRI_orig = (AST / ULN_AST) / (PLT / 100)
  )
df_apri <- adjust_outliers(df_apri, "APRI_orig")
summary(df_apri)

# Normalize APRI score (0 to 1)
df_apri <- df_apri %>%
  mutate(
    APRI = (APRI_orig - min(APRI_orig, na.rm = TRUE)) / (max(APRI_orig, na.rm = TRUE) - min(APRI_orig, na.rm = TRUE))
  )

# Summary of APRI scores
summary(df_apri$APRI)

## NFS Score

In [ ]:
df_nfs <- df_all %>% select(eid, "person_id_year", SEX, BMI, AGE, DM, AST = `Aspartate aminotransferase`,
                     ALT = `Alanine aminotransferase`,
                     PLT = `Platelet count`,
                    Albumin)

df_nfs <- df_nfs %>%
  mutate(
    DM_numeric = as.numeric(DM),
    AST_ALT_ratio = ifelse(ALT == 0 | is.na(ALT), NA, AST / ALT),
    NFS_orig = -1.675 + 0.037 * AGE + 0.094 * BMI + 1.13 * ifelse(DM_numeric == 1, 1, 0) + 
          0.99 * AST_ALT_ratio - 0.013 * PLT - 0.66 * Albumin
  )

summary(df_nfs$NFS_orig)
range(df_nfs$NFS_orig, na.rm = TRUE)
unique(df_nfs$NFS_orig)

df_nfs$NFS_orig[is.infinite(df_nfs$NFS_orig)] <- NA

# Normalize NFS score (0 to 1)
df_nfs <- df_nfs %>%
  mutate(
    NFS = (NFS_orig - min(NFS_orig, na.rm = TRUE)) / (max(NFS_orig, na.rm = TRUE) - min(NFS_orig, na.rm = TRUE))
  )

# Summary of NFS scores
summary(df_nfs$NFS)
head(df_nfs)

In [ ]:
check_zero_to_one <- function(x, column_name) {
  if (!is.numeric(x)) {
    cat(column_name, "is not numeric.\n")
    return(FALSE)
  }
  
  values_in_range <- x >= 0 & x <= 1
  all_in_range <- all(values_in_range, na.rm = TRUE)
  
  cat(column_name, "check:\n")
  cat("  All values between 0 and 1:", all_in_range, "\n")
  cat("  Number of NA values:", sum(is.na(x)), "\n")
  
  if (!all_in_range) {
    out_of_range <- x[!values_in_range & !is.na(x)]
    cat("  Values out of range:", out_of_range, "\n")
  }
  
  cat("\n")
  return(all_in_range)
}

# Check "status" column
status_check <- check_zero_to_one(df_amap$status, "status")

# Check "aMAP" column
amap_check <- check_zero_to_one(df_amap$aMAP, "aMAP")

# Overall result
if (status_check && amap_check) {
  cat("Both 'status' and 'aMAP' columns contain only values between 0 and 1.\n")
} else {
  cat("At least one column contains values outside the range 0 to 1 or is not numeric.\n")
}

## Benchmarks together

In [ ]:


df_benchmark <- df_amap %>%
  select(eid, person_id_year, aMAP) %>%
  inner_join(df_apri %>% select(person_id_year, APRI), by = "person_id_year") %>%
  inner_join(df_fib4 %>% select(person_id_year, FIB4), by = "person_id_year") %>%
  inner_join(df_nfs %>% select(person_id_year, NFS), by = "person_id_year") %>%
  inner_join(df_early_cirrhosis, by = "eid")

df_y <- df_y %>% select(person_id_year, status, year_diff, year_of_diag, year)

df_benchmark <- merge(df_benchmark, df_y, by="person_id_year") %>% filter(status !=2)

# Summary of the benchmark dataframe
summary(df_benchmark)
write_csv(df_benchmark, file="data/df_benchmark.csv")

In [ ]:
colnames(df_y)